causal conv와 dilation을 적용하지 않은 이유
- 시계열 예측 과제의 경우 인과성이 중요하지만 분류는 시계열 전체의 패턴을 학습하는 데 초점을 맞추기 때문에 모든 시점의 정보를 활용하는 것이 효과적
- 대형 커널을 사용해서 넓은 수용 영역을 커버 -> dilation을 적용할 필요 없음

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import math
from layers.RevIN import RevIN
from models.ModernTCN_Layer import series_decomp, Flatten_Head

## LayerNorm
- B: batch
- M: 배치 안의 추가적인 그룹 차원 (변수 개수, patch, window 등)
- D: feature/channel (LayerNorm이 적용되는 대상)
- N: 시계열 길이 (sequence length, time steps)
- Layernorm()은 마지막 차원을 정규화 대상으로 하기 때문에 permute()를 통해 마지막 차원이 feature(=D)가 되도록 바꿈
- (B*M, N, D)에서 각 (N, D) 조각마다 feature 차원(D)에 대해 정규화가 이루어짐

In [ ]:
class LayerNorm(nn.Module):

    def __init__(self, channels, eps=1e-6, data_format="channels_last"):
        super(LayerNorm, self).__init__()
        self.norm = nn.Layernorm(channels)

    def forward(self, x):

        B, M, D, N = x.shape
        x = x.permute(0, 1, 3, 2)
        x = x.reshape(B * M, N, D)
        x = self.norm(
            x)
        x = x.reshape(B, M, N, D)
        x = x.permute(0, 1, 3, 2)
        return x

In [ ]:
def get_conv1d(in_channels, out_channels, kernel_size, stride, padding, dilation, groups, bias):
    return nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride,
                     padding=padding, dilation=dilation, groups=groups, bias=bias)


def get_bn(channels):
    return nn.BatchNorm1d(channels)

def conv_bn(in_channels, out_channels, kernel_size, stride, padding, groups, dilation=1,bias=False):
    if padding is None:
        padding = kernel_size // 2
    result = nn.Sequential()
    result.add_module('conv', get_conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size,
                                         stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias))
    result.add_module('bn', get_bn(out_channels))
    return result

## fuse_bn()
- BN을 포함한 Conv는 새로운 Conv 하나로 대체할 수 있음
- BN 식을 Conv 출력에 직접 대입해 weight와 bias를 다시 정의
- 추론 시 BN 레이어를 제거할 수 있어서 속도와 메모리 효율이 개선됨
- 반환값은 새로운 Conv 파라미터인 fused_weight, fused_bias

In [ ]:
def fuse_bn(conv, bn):

    kernel = conv.weight
    running_mean = bn.running_mean
    running_var = bn.running_var
    gamma = bn.weight
    beta = bn.bias
    eps = bn.eps
    std = (running_var + eps).sqrt()
    t = (gamma / std).reshape(-1, 1, 1)
    return kernel * t, beta - running_mean * gamma / std

## ReparamLargeKernelConv
 structural re-parameterization
 - 큰 커널을 작은 커널과 함께 학습하다가 추론 시에 하나의 Conv로 합쳐서 연산 효율을 높임
 - Conv는 선형 연산이므로 weight와 bias를 합쳐줄 수 있음
- RepLKNet 같은 대형 커널 기반 네트워크에서 쓰인 아이디어

`__init__`
- lkb_origin: 큰 커널 Conv + BN
- small_conv: 작은 커널 Conv + BN (선택적, None이면 없음)
- lkb_reparam: 추론 시 사용할 단일 Conv (처음엔 없음)
- small_kernel_merged=True라면 학습할 때도 이미 합쳐진 Conv를 쓰도록 설정

`forward`
- 추론 모드에서 merge_kernel()을 이미 실행했다면 → lkb_reparam 하나만 실행
- 학습 모드라면 → 큰 커널 conv + 작은 커널 conv를 동시에 실행하고 결과를 합산

`PaddingTwoEdge1d()`
- 작은 커널을 큰 커널 크기로 맞추기 위해 좌우에 패딩 추가
- x는 Conv weight, shape은 (out_channels, in_channels, kernel_size)


`get_equivalent_kernel_bias`
- Conv+BN → Conv로 바꾼 후 large/small conv를 합산
1. 큰 커널 Conv+BN → fuse_bn()으로 합침 → (eq_k, eq_b)
2. 작은 커널도 Conv+BN 합침 → (small_k, small_b)
3. bias는 그냥 더하고 kernel은 작은 커널을 패딩으로 확장한 후 더함
- 최종적으로 large, small을 합친 kernel, bias를 반환

`merge_kernel`
- 추론용 Conv(큰 커널, 작은 커널 합친 것)으로 변환
- weight는 nn.Parameter, nn.Parameter의 .data 속성에 직접 할당하면 gradient 추적을 거치지 않고 바로 값을 바꿈 (추론 단계여서 gradient는 필요 없음)
- `__delattr__`: 기존 1kb_origin, small_conv는 삭제





In [ ]:
class ReparamLargeKernelConv(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size,
                 stride, groups,
                 small_kernel,
                 small_kernel_merged=False, nvars=7):
        super(ReparamLargeKernelConv, self).__init__()
        self.kernel_size = kernel_size
        self.small_kernel = small_kernel
        # We assume the conv does not change the feature map size, so padding = k//2. Otherwise, you may configure padding as you wish, and change the padding of small_conv accordingly.
        padding = kernel_size // 2
        if small_kernel_merged:
            self.lkb_reparam = nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size,
                                         stride=stride, padding=padding, dilation=1, groups=groups, bias=True)
        else:
            self.lkb_origin = conv_bn(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size,
                                        stride=stride, padding=padding, dilation=1, groups=groups,bias=False)
            if small_kernel is not None:
                assert small_kernel <= kernel_size, 'The kernel size for re-param cannot be larger than the large kernel!'
                self.small_conv = conv_bn(in_channels=in_channels, out_channels=out_channels,
                                            kernel_size=small_kernel,
                                            stride=stride, padding=small_kernel // 2, groups=groups, dilation=1,bias=False)


    def forward(self, inputs):

        if hasattr(self, 'lkb_reparam'):
            out = self.lkb_reparam(inputs)
        else:
            out = self.lkb_origin(inputs)
            if hasattr(self, 'small_conv'):
                out += self.small_conv(inputs)
        return out

    def PaddingTwoEdge1d(self,x,pad_length_left,pad_length_right,pad_values=0):

        D_out,D_in,ks=x.shape
        if pad_values ==0:
            pad_left = torch.zeros(D_out,D_in,pad_length_left)
            pad_right = torch.zeros(D_out,D_in,pad_length_right)
        else:
            pad_left = torch.ones(D_out, D_in, pad_length_left) * pad_values
            pad_right = torch.ones(D_out, D_in, pad_length_right) * pad_values
        x = torch.cat([pad_left,x],dims=-1)
        x = torch.cat([x,pad_right],dims=-1)
        return x

    def get_equivalent_kernel_bias(self):
        eq_k, eq_b = fuse_bn(self.lkb_origin.conv, self.lkb_origin.bn)
        if hasattr(self, 'small_conv'):
            small_k, small_b = fuse_bn(self.small_conv.conv, self.small_conv.bn)
            eq_b += small_b
            eq_k += self.PaddingTwoEdge1d(small_k, (self.kernel_size - self.small_kernel) // 2,
                                          (self.kernel_size - self.small_kernel) // 2, 0)
        return eq_k, eq_b

    def merge_kernel(self):
        eq_k, eq_b = self.get_equivalent_kernel_bias()
        self.lkb_reparam = nn.Conv1d(in_channels=self.lkb_origin.conv.in_channels,
                                     out_channels=self.lkb_origin.conv.out_channels,
                                     kernel_size=self.lkb_origin.conv.kernel_size, stride=self.lkb_origin.conv.stride,
                                     padding=self.lkb_origin.conv.padding, dilation=self.lkb_origin.conv.dilation,
                                     groups=self.lkb_origin.conv.groups, bias=True)
        self.lkb_reparam.weight.data = eq_k
        self.lkb_reparam.bias.data = eq_b
        self.__delattr__('lkb_origin')
        if hasattr(self, 'small_conv'):
            self.__delattr__('small_conv')

## Block
`convffn1`
- groups=nvars로 설정
`convffn2`
- groups=dmoel로 설정

### **reshape을 이해해보자!**
- B: batch, M: 변수 개수, D: dmodel(각 변수의 feature 차원), N: sequence length
- Conv1d, BatchNorm은 (batch, channels, length)를 받기를 기대함
#### 1. depthwise large-kernel conv

```
x = x.reshape(B,M*D,N)
x = self.dw(x)
```
- 목표: 각 변수-특징 조합마다 독립적으로 1D conv 적용
- M*D개의 channel을 독립적으로 convolve -> depthwise conv

#### 2. BatchNorm
```
x = x.reshape(B*M, D, N)
self.norm(x)
```
- 목표: 각 변수(M)별로 독립적인 정규화
- BatchNorm은 channels=D를 정규화
- 변수 M개를 batch 차원에 흡수시킴으로써 변수마다 D차원 feature를 정규화할 수 있음

#### 3. FFN1
```
x = x.reshape(B, M*D, N)
self.ffn1pw1(x)
```
- 목표: 변수별 독립 FFN
- M*D를 채널로 묶고 groups=M(nvar)으로 지정
- 각 그룹 크기는 D -> 변수별로 D차원 묶음을 독립적으로 처리

#### 4. FFN2
```
x = x.permute(0, 2, 1, 3)   # (B, D, M, N)
x = x.reshape(B, D*M, N)
self.ffn2pw1(x)
```
- 목표: feature별 cross-variable 연산 (변수 간 상관관계 학습, MTCN의 핵심)
- D*M을 채널로 묶고 groups=D로 설정
- 각 그룹 크기는 M, 같은 feature index(D) 안에서 변수 M개를 묶어 연산

#### 5. 마지막에 residual connection

### **GELU()**
- Gaussian Error Linear Unit
- ReLU와 유사하지만 음수에서 기울기가 조금 있고, 양수 부분도 선형 증가가 아닌 약간 곡선 형식
- ReLU보다 부드러워서 gradient 흐름이 원할
- Transformer 같은 대규모 모델에서 안정성과 성능이 좋음
- 언어/시계열/연속 패턴 데이터에서 성능 우위


In [ ]:
class Block(nn.Module):
    def __init__(self, large_size, small_size, dmodel, dff, nvars, small_kernel_merged=False, drop=0.1):

        super(Block, self).__init__()
        self.dw = ReparamLargeKernelConv(in_channels=nvars * dmodel, out_channels=nvars * dmodel,
                                         kernel_size=large_size, stride=1, groups=nvars * dmodel,
                                         small_kernel=small_size, small_kernel_merged=small_kernel_merged, nvars=nvars)
        self.norm = nn.BatchNorm1d(dmodel)

        #convffn1
        self.ffn1pw1 = nn.Conv1d(in_channels=nvars * dmodel, out_channels=nvars * dff, kernel_size=1, stride=1,
                                 padding=0, dilation=1, groups=nvars)
        self.ffn1act = nn.GELU()
        self.ffn1pw2 = nn.Conv1d(in_channels=nvars * dff, out_channels=nvars * dmodel, kernel_size=1, stride=1,
                                 padding=0, dilation=1, groups=nvars)
        self.ffn1drop1 = nn.Dropout(drop)
        self.ffn1drop2 = nn.Dropout(drop)

        #convffn2
        self.ffn2pw1 = nn.Conv1d(in_channels=nvars * dmodel, out_channels=nvars * dff, kernel_size=1, stride=1,
                                 padding=0, dilation=1, groups=dmodel)
        self.ffn2act = nn.GELU()
        self.ffn2pw2 = nn.Conv1d(in_channels=nvars * dff, out_channels=nvars * dmodel, kernel_size=1, stride=1,
                                 padding=0, dilation=1, groups=dmodel)
        self.ffn2drop1 = nn.Dropout(drop)
        self.ffn2drop2 = nn.Dropout(drop)

        self.ffn_ratio = dff//dmodel
    def forward(self,x):

        input = x
        B, M, D, N = x.shape
        x = x.reshape(B,M*D,N)
        x = self.dw(x)
        x = x.reshape(B,M,D,N)
        x = x.reshape(B*M,D,N)
        x = self.norm(x)
        x = x.reshape(B, M, D, N)
        x = x.reshape(B, M * D, N)

        x = self.ffn1drop1(self.ffn1pw1(x))
        x = self.ffn1act(x)
        x = self.ffn1drop2(self.ffn1pw2(x))
        x = x.reshape(B, M, D, N)

        x = x.permute(0, 2, 1, 3)
        x = x.reshape(B, D * M, N)
        x = self.ffn2drop1(self.ffn2pw1(x))
        x = self.ffn2act(x)
        x = self.ffn2drop2(self.ffn2pw2(x))
        x = x.reshape(B, D, M, N)
        x = x.permute(0, 2, 1, 3)

        x = input + x
        return x


## Stage
- block을 순차적으로 쌓아주는 역할
- dff: FFN의 내부 확장 차원 역할
- 복잡한 패턴 학습 후 원래 차원으로 압축

In [ ]:
class Stage(nn.Module):
    def __init__(self, ffn_ratio, num_blocks, large_size, small_size, dmodel, dw_model, nvars,
                 small_kernel_merged=False, drop=0.1):

        super(Stage, self).__init__()
        d_ffn = dmodel * ffn_ratio
        blks = []
        for i in range(num_blocks):
            blk = Block(large_size=large_size, small_size=small_size, dmodel=dmodel, dff=d_ffn, nvars=nvars, small_kernel_merged=small_kernel_merged, drop=drop)
            blks.append(blk)

        self.blocks = nn.ModuleList(blks)

    def forward(self, x):

        for blk in self.blocks:
            x = blk(x)

        return x

## ModernTCN
### `__init__`
1. RevIN 정규화 사용 가능

2. Stem
- 입력 시계열을 patch_size 단위로 자름

3. stage 사이마다 dowmsample
4. stage마다 num_blocks개의 block을 쌓음 -> 각 block은 ReparamLargeKernelConv + FFN1 + FFN2 + Residual
5. head
- use_multi_scale=True면 모든 patch를 flatten해서 head에 넣음, patch 단위 정보 전체를 보존 -> 다양한 시간 범위 패턴 학습
- False면, downsampling을 통해 줄어든 patch 개수에 맞춰 flatten해서 head에 넣음 -> 계산량은 줄지만 정보 손실
6. classification 과제면 Linear를 마지막에 붙임

### `forward_feature`
- 입력 시계열 -> 패치 -> 다운샘플링 -> 여러 stage 통과 -> feature map
1. (B, M, L) -> (B, M, 1, L) 변환
- 1은 channel 축처럼 사용됨 (conv1d 맞추기용)
2. stage별로 downsample (Conv1d) 적용, 필요 시 padding
3. Stage(Block stack) 통과
4. 최종 (B, M, D, N) feature map 반환

### `structural_reparam`
- merge_kernel() 메소드를 가진 모듈(=ReparamLargeKernelConv)만 골라서 실행
- m.merge_kernel(): 학습 시 분리돼 있던 여러 Conv + BN을 하나의 Conv로 합침

In [ ]:
class ModernTCN(nn.Module):
    def __init__(self,task_name,patch_size,patch_stride, stem_ratio, downsample_ratio, ffn_ratio, num_blocks, large_size, small_size, dims, dw_dims,
                 nvars, small_kernel_merged=False, backbone_dropout=0.1, head_dropout=0.1, use_multi_scale=True, revin=True, affine=True,
                 subtract_last=False, freq=None, seq_len=512, c_in=7, individual=False, target_window=96, class_drop=0.,class_num = 10):

        super(ModernTCN, self).__init__()

        self.task_name = task_name
        self.class_drop = class_drop
        self.class_num = class_num


        # RevIN
        self.revin = revin
        if self.revin: self.revin_layer = RevIN(c_in, affine=affine, subtract_last=subtract_last)

        # stem layer & down sampling layers
        self.downsample_layers = nn.ModuleList()
        stem = nn.Sequential(
            nn.Conv1d(1, dims[0], kernel_size=patch_size, stride=patch_stride),
            nn.BatchNorm1d(dims[0])
        )
        self.downsample_layers.append(stem)

        self.num_stage = len(num_blocks)
        if self.num_stage > 1:
            for i in range(self.num_stage - 1):
                downsample_layer = nn.Sequential(
                    nn.BatchNorm1d(dims[i]),
                    nn.Conv1d(dims[i], dims[i + 1], kernel_size=downsample_ratio, stride=downsample_ratio),
                )
                self.downsample_layers.append(downsample_layer)

        self.patch_size = patch_size
        self.patch_stride = patch_stride
        self.downsample_ratio = downsample_ratio

        # backbone
        self.num_stage = len(num_blocks)
        self.stages = nn.ModuleList()
        for stage_idx in range(self.num_stage):
            layer = Stage(ffn_ratio, num_blocks[stage_idx], large_size[stage_idx], small_size[stage_idx], dmodel=dims[stage_idx],
                          dw_model=dw_dims[stage_idx], nvars=nvars, small_kernel_merged=small_kernel_merged, drop=backbone_dropout)
            self.stages.append(layer)


        # head
        patch_num = seq_len // patch_stride
        self.n_vars = c_in
        self.individual = individual
        d_model = dims[self.num_stage-1]


        if use_multi_scale:
            self.head_nf = d_model * patch_num
            self.head = Flatten_Head(self.individual, self.n_vars, self.head_nf, target_window,
                                     head_dropout=head_dropout)
        else:
            if patch_num % pow(downsample_ratio,(self.num_stage - 1)) == 0:
                self.head_nf = d_model * patch_num // pow(downsample_ratio,(self.num_stage - 1))
            else:
                self.head_nf = d_model * (patch_num // pow(downsample_ratio, (self.num_stage - 1))+1)


            self.head = Flatten_Head(self.individual, self.n_vars, self.head_nf, target_window,
                                     head_dropout=head_dropout)

        if self.task_name == 'classification':
            self.act_class = F.gelu
            self.class_dropout = nn.Dropout(self.class_drop)

            self.head_class = nn.Linear(self.n_vars[0]*self.head_nf,self.class_num)


    def forward_feature(self, x, te=None):

        B,M,L=x.shape

        x = x.unsqueeze(-2) # (B, M, 1, L)

        for i in range(self.num_stage):
            B, M, D, N = x.shape
            x = x.reshape(B * M, D, N) # Conv1D 입력 형식(batch, channels, length)에 맞춰서 변환
            if i==0: # 첫 stage
                if self.patch_size != self.patch_stride: # patch_stride < patch_size인 경우 마지막에 patch를 하나 더 만들 수 있게 padding
                    # stem layer padding
                    pad_len = self.patch_size - self.patch_stride
                    pad = x[:,:,-1:].repeat(1,1,pad_len)
                    x = torch.cat([x,pad],dim=-1)
            else: # 이후 stage
                if N % self.downsample_ratio != 0: # downsample conv의 stride 때문에 길이가 안 나눠떨어질 수 있음 -> 마지막 구간 패딩
                    pad_len = self.downsample_ratio - (N % self.downsample_ratio)
                    x = torch.cat([x, x[:, :, -pad_len:]],dim=-1)
            x = self.downsample_layers[i](x)
            _, D_, N_ = x.shape
            x = x.reshape(B, M, D_, N_)
            x = self.stages[i](x)
        return x

    def classification(self,x):

        x =  self.forward_feature(x,te=None)
        x = self.act_class(x)
        x = self.class_dropout(x)
        x = x.reshape(x.shape[0], -1)
        x = self.head_class(x)
        return x


    def forward(self, x, te=None):

        if self.task_name == 'classification':
            x = self.classification(x)

        return x



    def structural_reparam(self):
        for m in self.modules():
            if hasattr(m, 'merge_kernel'):
                m.merge_kernel()

## Model
forward의 인자
- x: 입력 시계열 데이터 (실제로 쓰이는 유일한 값)
- x_mark_enc: 인코더용 시간 인코딩 (호환성용, 여기선 안 씀)
- x_dec / x_mark_dec: 디코더 입력/시간 인코딩 (여기선 안 씀)
- mask: attention 모델용 mask (여기선 안 씀)
- te: temporal encoding (현재는 None으로 비활성화)
- forecasting, seq2seq 모델 전반에서 쓰이는 인터페이스를 맞추기 위해 들어있는 인자

In [ ]:
class Model(nn.Module):
    def __init__(self, configs):
        super(Model, self).__init__()
        # hyper param
        self.task_name = configs.task_name
        self.stem_ratio = configs.stem_ratio
        self.downsample_ratio = configs.downsample_ratio
        self.ffn_ratio = configs.ffn_ratio
        self.num_blocks = configs.num_blocks
        self.large_size = configs.large_size
        self.small_size = configs.small_size
        self.dims = configs.dims
        self.dw_dims = configs.dw_dims

        self.nvars = configs.enc_in
        self.small_kernel_merged = configs.small_kernel_merged
        self.drop_backbone = configs.dropout
        self.drop_head = configs.head_dropout
        self.use_multi_scale = configs.use_multi_scale
        self.revin = configs.revin
        self.affine = configs.affine
        self.subtract_last = configs.subtract_last

        self.freq = configs.freq
        self.seq_len = configs.seq_len
        self.c_in = self.nvars,
        self.individual = configs.individual
        self.target_window = configs.pred_len

        self.kernel_size = configs.kernel_size
        self.patch_size = configs.patch_size
        self.patch_stride = configs.patch_stride

        #classification
        self.class_dropout = configs.class_dropout
        self.class_num = configs.num_class


        # decomp
        self.decomposition = configs.decomposition


        self.model = ModernTCN(task_name=self.task_name,patch_size=self.patch_size, patch_stride=self.patch_stride, stem_ratio=self.stem_ratio,
                           downsample_ratio=self.downsample_ratio, ffn_ratio=self.ffn_ratio, num_blocks=self.num_blocks,
                           large_size=self.large_size, small_size=self.small_size, dims=self.dims, dw_dims=self.dw_dims,
                           nvars=self.nvars, small_kernel_merged=self.small_kernel_merged,
                           backbone_dropout=self.drop_backbone, head_dropout=self.drop_head,
                           use_multi_scale=self.use_multi_scale, revin=self.revin, affine=self.affine,
                           subtract_last=self.subtract_last, freq=self.freq, seq_len=self.seq_len, c_in=self.c_in,
                           individual=self.individual, target_window=self.target_window,
                            class_drop = self.class_dropout, class_num = self.class_num)

    def forward(self, x, x_mark_enc, x_dec, x_mark_dec, mask=None):
        x = x.permute(0, 2, 1)
        te = None
        x = self.model(x, te)
        return x